In [1]:
!pip install music21

    100% |████████████████████████████████| 18.0MB 86kB/s 
  Running setup.py bdist_wheel for music21 ... done
  Stored in directory: /root/.cache/pip/wheels/53/8b/a6/be1921c60a68f0bea31c6b6a0a7b125badd61294d6a694407f
Successfully built music21
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import keras
from keras import objectives, backend as K
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda
from keras.layers import LSTM, CuDNNLSTM, RepeatVector, TimeDistributed, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np
import os
import glob
import pickle
from music21 import converter, instrument, note, chord, stream

Using TensorFlow backend.


In [3]:
MAX_LENGTH = 300

In [7]:
class VAE(object):
    def __init__(self, vocab_size=500, max_length=300, latent_rep_size=64):
        self.encoder = None
        self.decoder = None
        self.autoencoder = None

        x = Input(shape=(max_length, vocab_size))

        vae_loss, encoded = self._build_encoder(x, latent_rep_size=latent_rep_size, max_length=max_length)
        self.encoder = Model(inputs=x, outputs=encoded)
        encoder_out = self.encoder(x)

        encoded_input = Input(shape=(latent_rep_size,))

        decoded = self._build_decoder(encoded_input, vocab_size, max_length)
        self.decoder = Model(encoded_input, decoded)
        
        decoder_out = self.decoder(encoder_out)

        self.autoencoder = Model(inputs=x, outputs=decoder_out)
        self.autoencoder.compile(optimizer='Adam',
                                 loss=vae_loss,
                                 metrics=['accuracy'])
        
    def _build_encoder(self, x, latent_rep_size=64, max_length=300, epsilon_std=0.01):
        h = CuDNNLSTM(512, return_sequences=False, name='lstm_1')(x)
        h = Dropout(0.2)(h)
        h = Dense(256, activation='relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev=epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation='linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation='linear')(h)
    
        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _build_decoder(self, encoded, vocab_size, max_length):
        repeated_context = RepeatVector(max_length)(encoded)
    
        h = CuDNNLSTM(512, return_sequences=True, name='dec_lstm_1')(repeated_context)
    
        decoded = TimeDistributed(Dense(vocab_size, activation='softmax'), name='decoded_mean')(h)
    
        return decoded


In [15]:
def parse_midi_files(dir):
    notes = []
    songs = []

    for file in glob.glob(os.path.join(dir, '*.mid')):
        song = []
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        songs.append(song)
        notes += song

    return notes, songs

In [16]:
notes, songs = parse_midi_files('/root/userspace/keras-composer/midi_songs')

Parsing /root/userspace/keras-composer/midi_songs/bwv782.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv773.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv786.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv777.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv784.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv775.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv780.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv779.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv783.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv774.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv778.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv785.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv776.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv781.mid
Parsing /root/userspace/keras-composer/midi_songs/bwv772.mid


In [17]:
pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict([[number, note] for note, number in note_to_int.items()])

encoded_songs = [[note_to_int[note] for note in song] for song in songs]

padded_songs = pad_sequences(encoded_songs, maxlen=MAX_LENGTH)

In [18]:
temp = np.zeros((padded_songs.shape[0], MAX_LENGTH, n_vocab))
temp[np.expand_dims(np.arange(padded_songs.shape[0]), axis=0).reshape(padded_songs.shape[0], 1), 
           np.repeat(np.array([np.arange(MAX_LENGTH)]), padded_songs.shape[0], axis=0), padded_songs] = 1

one_hot_encoded_songs = temp

In [19]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + \
               model_name + "-{epoch:02d}-{acc:.2f}-{loss:.2f}.h5"
    directory = os.path.dirname(filepath)

    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

    checkpointer = ModelCheckpoint(filepath=filepath,
                                                              monitor='loss',
                                                              verbose=1,
                                                              save_best_only=True)

    return checkpointer

In [20]:
model = VAE(vocab_size=n_vocab, latent_rep_size=2, max_length=MAX_LENGTH)

In [21]:
checkpointer = create_model_checkpoint('/root/userspace/keras-composer/vae_output', 'music_vae')
csv_logger = CSVLogger(os.path.join('/root/userspace/keras-composer/vae_output', 'music_vae_log.csv'))

history = model.autoencoder.fit(x=one_hot_encoded_songs, 
                                                        y=one_hot_encoded_songs,
                                                        batch_size=1, 
                                                        epochs=10000, 
                                                        callbacks=[checkpointer, csv_logger], 
                                                        verbose=0)

Epoch 00001: loss improved from inf to 13.05794, saving model to /root/userspace/keras-composer/vae_output/music_vae-01-0.03-13.06.h5
Epoch 00002: loss improved from 13.05794 to 12.54065, saving model to /root/userspace/keras-composer/vae_output/music_vae-02-0.03-12.54.h5
Epoch 00003: loss improved from 12.54065 to 12.33448, saving model to /root/userspace/keras-composer/vae_output/music_vae-03-0.04-12.33.h5
Epoch 00004: loss improved from 12.33448 to 12.29151, saving model to /root/userspace/keras-composer/vae_output/music_vae-04-0.04-12.29.h5
Epoch 00005: loss improved from 12.29151 to 12.28153, saving model to /root/userspace/keras-composer/vae_output/music_vae-05-0.04-12.28.h5
Epoch 00006: loss did not improve
Epoch 00007: loss did not improve
Epoch 00008: loss improved from 12.28153 to 12.27743, saving model to /root/userspace/keras-composer/vae_output/music_vae-08-0.05-12.28.h5
Epoch 00009: loss improved from 12.27743 to 12.25689, saving model to /root/userspace/keras-composer/va

KeyboardInterrupt: 

In [22]:
model.autoencoder.load_weights('/root/userspace/keras-composer/vae_output/music_vae-827-0.48-5.67.h5')

In [34]:
prediction_output = model.autoencoder.predict(one_hot_encoded_songs[np.newaxis, 0])

In [35]:
prediction_indices = np.argmax(prediction_output, axis=2)

prediction_song = [int_to_note[index] for index in prediction_indices[0]]
print(prediction_song)

['5.11', '1.4', 'F4', 'G4', 'D5', 'G#4', 'C5', 'D5', 'E3', 'C5', 'D5', 'G4', 'C#5', 'D4', 'D5', 'E5', 'D5', 'E-4', 'E-5', 'F5', 'E-5', 'F4', 'D5', 'E-5', 'G3', 'D5', 'F4', 'E-4', 'B-4', 'A4', 'E-4', 'C5', 'F3', 'E-3', 'D3', 'E-3', 'D5', 'C3', 'E4', 'D5', '8.0', '7.10', 'C5', 'G#4', 'F3', 'A4', 'G3', 'F3', 'D5', 'E-3', 'C5', 'E-5', 'F3', 'C5', 'E-5', '11.2', '9.0', '3.9', 'B4', 'A4', 'B2', '9.0', '11.2', '0.3', 'F4', 'G3', 'E-5', 'E-3', '11.2', 'G#3', '11.2', '0.3', 'G3', '3.7', '8.0', 'F3', 'G5', 'A2', 'E-3', 'G2', 'B4', 'E-3', 'F3', '5.8', 'G2', '2.5', 'E-3', 'C5', '7.10', 'D3', 'C5', 'G#2', 'B-2', 'A2', 'A2', 'D5', 'A4', '7.10', '5.9', '3.7', '7.9', 'F4', 'G3', 'F3', 'B2', 'C#5', 'F5', '0.3', 'C#3', '9.0', 'B-2', 'C#5', 'C#3', 'C5', 'C#5', 'F3', 'E-5', 'F#5', 'A3', 'C3', 'B2', 'D5', '9.0', 'B-3', 'B-4', 'G#4', 'F4', 'G#3', 'C3', 'B-5', 'F3', 'G3', 'G#5', 'E-3', 'G5', 'D4', '1.3', '0.1', 'C5', 'C5', 'C5', 'C5', 'C4', '0', 'C5', '0.3', '0.3', 'E-5', 'E-5', 'D5', 'G#3', 'C#4', 'B-3', 'G

In [36]:
def create_midi(prediction_output, file_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=file_path)

In [37]:
create_midi(prediction_song, '/root/userspace/keras-composer/test_vae_onehot_out_20180929_1.midi')

In [38]:
decoder_input = np.array([[-0.0001, 0.0001]])
decoder_predicted = model.decoder.predict(decoder_input)

In [39]:
prediction_indices = np.argmax(decoder_predicted, axis=2)

prediction_song = [int_to_note[index] for index in prediction_indices[0]]
print(prediction_song)

['D4', 'E4', 'A3', 'C#5', '0.4', '4.7', 'C#5', 'A4', 'G4', '9.2', 'B3', 'F#5', 'G5', 'G3', '11.2', '11.2', 'F#3', 'D5', 'G4', 'A3', 'F#3', 'D3', 'C4', 'C4', 'F#3', '9.0', 'C4', 'B4', 'C5', 'D5', 'G3', 'B4', 'G4', 'D5', 'B4', 'F#5', 'F5', 'C3', 'E5', 'C5', 'A4', 'A5', '2.4', '0.4', '6.9', '7.11', '9.0', 'B2', 'E2', 'A2', 'B2', 'B4', 'G5', 'G5', 'E5', '11.2', '4.7', '6.9', '7.11', 'C#3', 'G5', 'E5', 'D5', 'C3', 'C5', 'D5', 'E3', 'C5', 'D5', 'A3', 'C5', 'D5', 'E3', 'C5', 'D5', 'C5', 'G#3', 'D5', 'C5', 'B2', 'D5', 'D5', 'B2', 'D5', 'G4', 'C3', 'D5', 'C5', 'D5', 'D3', 'C5', 'G5', 'F#3', 'E3', 'C5', 'A3', 'C5', 'D5', 'D4', 'C5', 'D5', 'C5', 'A3', 'D5', 'C5', 'F#3', 'D5', 'C5', 'D3', 'D5', 'B2', 'E3', 'D5', 'C5', 'D5', 'D5', 'C5', 'B4', 'G3', 'C5', 'B4', 'B3', 'C5', 'B4', 'D4', 'C5', 'B4', 'G4', 'C5', 'B4', 'D4', 'C5', 'B4', 'C5', 'B4', 'B4', 'B4', 'B4', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B3', 'C4', 'B4', 'E-5', 'C5', 'D5', 'G#3', 'C#4', 'B4', 'G3', 'G#3', '0.4', '9.0', '6.9', 'C5', '0.6', 

In [40]:
create_midi(prediction_song, '/root/userspace/keras-composer/test_vae_onehot_out_20180929_2.midi')